## LangGraph Chatbot with Tools

In [ ]:
%pip install langgraph langsmith langchain langchain_community langchain-ollama python-dotenv

In [2]:
from typing import Annotated
from typing_extensions import TypedDict

In [ ]:
%pip install arxiv wikipedia

In [5]:
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=300)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)

wikipedia_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)
wikipedia_tool = WikipediaQueryRun(api_wrapper=wikipedia_wrapper)

In [8]:
wikipedia_tool.invoke("Who is Lionel Messi?")

'Page: Messi–Ronaldo rivalry\nSummary: The Messi–Ronaldo rivalry, or Ronaldo–Messi rivalry, is a sporting rivalry in football propelled by the media and fans that involves Argentine footballer Lionel Messi and Portuguese footballer Cristiano Ronaldo, mainly for being contemporaries and due to their si'

In [9]:
arxiv_tool.invoke("attention is all you need")

"Published: 2024-07-22\nTitle: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models\nAuthors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervini\nSummary: The inference demand for LLMs has skyrocketed in recent months, and serving\nmodels with "

In [10]:
tools = [wikipedia_tool]

In [11]:
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

In [12]:
from langgraph.graph import StateGraph, START, END

In [13]:
graph_builder = StateGraph(State)

In [14]:
from langchain_ollama import ChatOllama

In [15]:
llm = ChatOllama(model="llama3.2")

In [17]:
llm_with_tools = llm.bind_tools(tools)

In [18]:
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state['messages'])]}

In [19]:
from langgraph.prebuilt import ToolNode, tools_condition

In [20]:
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("chatbot", END)

In [22]:
graph = graph_builder.compile()

In [23]:
user_input = "Hi there! My name is Tomas"

events = graph.stream(
    {"messages": [("user", user_input)]}, stream_mode="values"
)

for event in events:
    event['messages'][-1].pretty_print()

================================ Human Message =================================

Hi there! My name is Tomas
================================== Ai Message ==================================
Tool Calls:
  wikipedia (da1a17a2-8975-4ffb-9200-c09a1cabb348)
 Call ID: da1a17a2-8975-4ffb-9200-c09a1cabb348
  Args:
    query: Tomas


/Users/tomasdaureli/Desktop/Tomas/Getting Started with LangGraph /env/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/tomasdaureli/Desktop/Tomas/Getting Started with LangGraph /env/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


================================= Tool Message =================================
Name: wikipedia

No good Wikipedia Search Result was found
================================== Ai Message ==================================

It seems like I couldn't find any relevant information about a person named Tomas in my database. However, I can try to help you with a different question or topic.

If you're asking about the name Tomas in general, it's a popular name in many countries, including Spain, Portugal, and some Latin American nations. It's derived from the Germanic word "thoma," which means "twin." Could you please provide more context or information about who or what you're referring to as Tomas? I'll do my best to assist you!
